In [1]:
import ollama
import chromadb
from chromadb.utils import embedding_functions
import random
import re
import os
from pathlib import Path
from colorama import init, Fore, Back, Style
import simpleaudio as sa
import threading
import time

# Initialize colorama
init(autoreset=True)

# ========================
# Enhanced Audio System with Looping
# ========================
class AudioSystem:
    def __init__(self):
        self.sound_dir = Path("sounds")
        self.active_players = {}  # Track playing sounds
        self.loop_flags = {}      # Control looping
        
        # Map your specific filenames to tone categories
        self.sound_mapping = {
            "dark": "evil-lurks-258066.wav",
            "humorous": "funny-hiphop-party-notification-music-sound-334710.wav",
            "psychological": "anxiety-178909.wav",
            "neutral": "ghost-2-275952.wav"
        }
        
        # Verify files exist
        print("Sound files found:")
        for tone, filename in self.sound_mapping.items():
            path = self.sound_dir / filename
            print(f" - {tone}: {'✅' if path.exists() else '❌'} {filename}")

    def _play_loop(self, tone):
        """Background thread for looping audio"""
        path = self.sound_dir / self.sound_mapping[tone]
        while self.loop_flags.get(tone, False):
            try:
                wave_obj = sa.WaveObject.from_wave_file(str(path))
                self.active_players[tone] = wave_obj.play()
                while self.loop_flags.get(tone, False) and self.active_players[tone].is_playing():
                    time.sleep(0.1)
            except Exception as e:
                print(f"Audio error ({tone}): {e}")
                break

    def play(self, tone, loop=False):
        """Play sound with optional looping"""
        # Stop any existing playback
        self.stop(tone)
        
        # Set loop flag before starting
        self.loop_flags[tone] = loop
        
        if loop:
            # Start looping thread
            threading.Thread(
                target=self._play_loop,
                args=(tone,),
                daemon=True
            ).start()
        else:
            # Play once
            try:
                path = self.sound_dir / self.sound_mapping[tone]
                wave_obj = sa.WaveObject.from_wave_file(str(path))
                self.active_players[tone] = wave_obj.play()
            except Exception as e:
                print(f"Audio error: {e}")

    def stop(self, tone):
        """Stop specific sound"""
        if tone in self.active_players:
            self.active_players[tone].stop()
        self.loop_flags[tone] = False

# ========================
# Visual System (Unchanged)
# ========================
class VisualSystem:
    def __init__(self):
        self.styles = {
            "dark": {
                "text": Fore.LIGHTRED_EX,
                "bg": Back.BLACK,
                "border": "🕸️"
            },
            "humorous": {
                "text": Fore.CYAN,
                "bg": Back.LIGHTBLACK_EX,
                "border": "🤡"
            },
            "psychological": {
                "text": Fore.MAGENTA,
                "bg": Back.BLACK,
                "border": "🌀"
            },
            "neutral": {
                "text": Fore.WHITE,
                "bg": Back.BLACK,
                "border": "│"
            }
        }
    
    def format_text(self, text, tone):
        style = self.styles.get(tone, self.styles["neutral"])
        return f"{style['bg']}{style['text']}{text}{Style.RESET_ALL}"
    
    def get_border(self, tone):
        return self.styles.get(tone, self.styles["neutral"])["border"]

# ========================
# Game Core (Updated with Audio Control)
# ========================
class MysteryGame:
    def __init__(self):
        self.client = chromadb.Client()
        self.embed_func = embedding_functions.DefaultEmbeddingFunction()
        self.visual = VisualSystem()
        self.audio = AudioSystem()
        self._setup_game()

    def _setup_game(self):
        """Initialize game state"""
        self.state = {
            "setting": random.choice(["1920s mansion", "space station"]),
            "victim": random.choice(["Sir Reginald", "Captain Vega"]),
            "suspects": ["the butler", "the widow", "the journalist"],
            "solved": False,
            "score": 0,
            "turns_left": 30,
            "current_tone": "neutral"
        }
        
        # Start neutral background audio
        self.audio.play("neutral", loop=True)

    def run(self):
        """Main game loop"""
        print(self.visual.format_text(
            "🕵️‍♂️ AUTO-SLEUTH: NOTEBOOK EDITION 🕵️‍♀️",
            "neutral"
        ))
        
        while not self.state["solved"] and self.state["turns_left"] > 0:
            try:
                player_input = input(f"\n[{self.state['turns_left']} turns] > ").strip()
                
                if player_input.lower() == "quit":
                    break
                    
                self._process_turn(player_input)
                
            except KeyboardInterrupt:
                print("\nGame stopped!")
                break
                
        self._end_game()

    def _process_turn(self, player_input):
        """Handle a game turn"""
        self.state["turns_left"] -= 1
        
        # Analyze tone from input
        new_tone = self._detect_tone(player_input)
        if new_tone != self.state["current_tone"]:
            self.audio.stop(self.state["current_tone"])
            self.audio.play(new_tone, loop=True)
            self.state["current_tone"] = new_tone
        
        # Generate response
        response = ollama.generate(
            model="mistral",
            prompt=self._build_prompt(player_input)
        )
        
        # Display
        print(self._format_response(response["response"]))

    def _detect_tone(self, text):
        """Simple tone detection"""
        text = text.lower()
        if any(word in text for word in ["blood", "kill", "die"]):
            return "dark"
        elif any(word in text for word in ["lol", "joke", "funny"]):
            return "humorous"
        elif any(word in text for word in ["why", "dream", "real"]):
            return "psychological"
        return "neutral"

    def _build_prompt(self, player_input):
        return f"""
        [ROLE] Mystery Game AI
        [TONE] {self.state['current_tone']}
        [CONTEXT]
        Setting: {self.state['setting']}
        Victim: {self.state['victim']}
        Suspects: {', '.join(self.state['suspects'])}
        
        [PLAYER INPUT] {player_input}
        
        Respond with:
        1. Tone-appropriate narration
        2. CLUE: if relevant
        3. QUESTION: to continue interaction
        """

    def _format_response(self, raw_text):
        """Apply visual styling"""
        styled = self.visual.format_text(raw_text, self.state["current_tone"])
        border = self.visual.get_border(self.state["current_tone"])
        return f"{border*3}\n{styled}\n{border*3}"

    def _end_game(self):
        self.audio.stop(self.state["current_tone"])
        if self.state["solved"]:
            msg = f"🔎 Case Solved! Score: {self.state['score']}"
            tone = "dark"
        else:
            msg = "💀 The truth remains hidden..."
            tone = "neutral"
            
        print(self.visual.format_text(
            f"\n{'═'*50}\n{msg}\n{'═'*50}",
            tone
        ))

# ========================
# Notebook-Friendly Setup
# ========================
if __name__ == "__main__":
    # Create sounds directory if missing
    sounds_dir = Path("sounds")
    sounds_dir.mkdir(exist_ok=True)
    
    # Verify critical files exist
    required_files = {
        "evil-lurks-258066.wav",
        "funny-hiphop-party-notification-music-sound-334710.wav",
        "anxiety-178909.wav", 
        "ghost-2-275952.wav"
    }
    missing = required_files - {f.name for f in sounds_dir.glob("*") if f.is_file()}
    
    if missing:
        print(f"⚠️ Missing files: {missing}")
        print("Please add them to the 'sounds' folder")
    else:
        game = MysteryGame()
        game.run()

Sound files found:
 - dark: ✅ evil-lurks-258066.wav
 - humorous: ✅ funny-hiphop-party-notification-music-sound-334710.wav
 - psychological: ✅ anxiety-178909.wav
 - neutral: ✅ ghost-2-275952.wav
🕵️‍♂️ AUTO-SLEUTH: NOTEBOOK EDITION 🕵️‍♀️



[30 turns] >  f


│││
 In the cold, silent expanse of the space station, a mystery unfolds. Sir Reginald, a renowned astrophysicist, has met an untimely end. The butler, a man of few words but many secrets; the widow, grieving yet harboring a past shrouded in mystery; and the journalist, ever-eager for a sensational story - these are the suspects.

   You find yourself standing in front of Sir Reginald's quarters, where the tragedy occurred. The air is heavy with tension, the silence broken only by the hum of machinery. What do you want to do next?

   (A) Explore the crime scene
   (B) Interrogate a suspect
   (C) Search for clues or evidence
   (D) Leave the station immediately
│││

Game stopped!

══════════════════════════════════════════════════
💀 The truth remains hidden...
══════════════════════════════════════════════════
